<a href="https://colab.research.google.com/github/golderalex6/project/blob/main/Processing_VN_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --You need to execute this block first!!

In [1]:
import os
if os.path.exists('project'):
    print('You already have the folder!!')
else:
    !git clone https://github.com/golderalex6/project.git
    !ls /content/project/
    !unzip /content/project/Data.zip

Cloning into 'project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 26), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (50/50), 44.36 MiB | 35.21 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Data.zip  Main.ipynb  Processing_US_Data.ipynb	Processing_VN_Data.ipynb
Archive:  /content/project/Data.zip
  inflating: us_data/covid19-us(main).csv  
  inflating: us_data/covid19-us(reserve).csv  
  inflating: us_data/state-acronym.csv  
  inflating: us_data/us-state-areas.csv  
  inflating: us_data/us-state-population.xlsx  
   creating: vietnam_data/.ipynb_checkpoints/
  inflating: vietnam_data/.ipynb_checkpoints/tinh_thanh-checkpoint.txt  
  inflating: vietnam_data/cities-area.xlsx  
  inflating: vietnam_data/cities-population.xlsx  
  inflating: vietnam_data/covid19-vn(main).csv  
  inflating: vietnam_data/covid19-vn(main).ods  
  inflating: vietnam_data/tinh_t

In [3]:
import pandas as pd
from pandas import DataFrame
import numpy as np

In [ ]:
vn_raw=pd.read_csv('/content/vietnam_data/covid19-vn(main).csv',encoding='utf-8')
def change(j):
    u=j.split()[0]
    v=u.split('/')
    return '-'.join([v[2],v[0],v[1]])
vn_raw['date']=vn_raw['date'].map(change)
vn_filter=vn_raw[('2021-07-01'<=vn_raw['date'])&(vn_raw['date']<='2021-08-31')]
vn_raw=DataFrame(vn_filter.iloc[:,4:].sum()).reset_index().rename(columns={'index':'Province',0:'ncase_th_period'})

# Area

In [4]:
cities_area=pd.read_excel('/content/vietnam_data/cities-area.xlsx')
cities_area.columns=['Province','area']
cities_area.drop([0,1,2],inplace=True)
cities_area.reset_index(drop=True,inplace=True)
cities_area

,Province,area
0,An Giang,3536.83
1,Ba Ria - Vung Tau,1982.56
2,Bac Giang,3895.89
3,Bac Kan,4859.96
4,Bac Lieu,2667.88
...,...,...
58,Tra Vinh,2390.76
59,Tuyen Quang,5867.95
60,Vinh Long,1525.73
61,Vinh Phuc,1236


# Population

In [ ]:
cities_population=pd.read_excel('/content/vietnam_data/cities-population.xlsx')
cities_population.columns=['Province','Population']
cities_population.drop([0,1,2],inplace=True)
cities_population.reset_index(drop=True,inplace=True)
cities_population['Population']=cities_population['Population']*1000
cities_population['Population']=cities_population['Population'].astype(int)
cities_population

,Province,Population
0,An Giang,1909510
1,Ba Ria - Vung Tau,1176080
2,Bac Giang,1875240
3,Bac Kan,323710
4,Bac Lieu,918510
...,...,...
58,Tra Vinh,1018630
59,Tuyen Quang,801670
60,Vinh Long,1029020
61,Vinh Phuc,1191780


# Density

In [ ]:
vn_case_periods=pd.merge(vn_raw,cities_population,how='inner')
vn_case_periods=pd.merge(vn_case_periods,cities_area,how='inner')
vn_case_periods['Density(peo/sq km)']=vn_case_periods['Population']/vn_case_periods['area']
vn_case_periods

,Province,ncase_th_period,Population,area,Density(peo/sq km)
0,An Giang,1913,1909510,3536.83,539.893068
1,Ba Ria - Vung Tau,3463,1176080,1982.56,593.212816
2,Bac Giang,142,1875240,3895.89,481.338025
3,Bac Kan,0,323710,4859.96,66.607544
4,Bac Lieu,146,918510,2667.88,344.284601
...,...,...,...,...,...
58,Tra Vinh,1287,1018630,2390.76,426.069534
59,Tuyen Quang,1,801670,5867.95,136.61841
60,Vinh Long,2049,1029020,1525.73,674.444364
61,Vinh Phuc,136,1191780,1236,964.223301


# Case a day

In [ ]:
vn_day_2m=vn_filter.set_index('date')
vn_day_2m.rename_axis(None,inplace=True)
vn_day_2m=vn_day_2m.iloc[:,3:]
vn_day_2m=vn_day_2m.transpose()
vn_day_2m=vn_day_2m.reset_index().rename(columns={'index':'Province'})
vn_day_2m=vn_day_2m[vn_day_2m['Province'].isin(['Thai Binh','Quang Binh','Phu Yen','Bac Lieu','Hung Yen','Nam Dinh','Binh Thuan','Bac Giang','Son La','Hoa Binh'])]
vn_day_2m.set_index('Province',inplace=True)
vn_day_2m

,2021-07-01,2021-07-02,2021-07-03,2021-07-04,2021-07-05,2021-07-06,2021-07-07,2021-07-08,2021-07-09,2021-07-10,...,2021-08-22,2021-08-23,2021-08-24,2021-08-25,2021-08-26,2021-08-27,2021-08-28,2021-08-29,2021-08-30,2021-08-31
Province,,,,,,,,,,,,,,,,,,,,,
Bac Giang,5,2,0,3,7,5,5,9,10,4,...,6,9,3,0,2,2,1,0,0,0
Bac Lieu,0,0,3,0,0,0,0,2,2,1,...,0,5,7,7,4,3,13,12,3,5
Binh Thuan,0,0,0,0,0,0,0,1,0,0,...,47,36,56,106,48,87,49,78,46,59
Hoa Binh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hung Yen,11,3,10,0,9,6,5,2,8,7,...,0,0,0,0,1,0,0,0,2,0
Nam Dinh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Phu Yen,26,40,37,32,40,41,14,24,28,33,...,38,43,24,27,24,21,16,21,14,23
Quang Binh,0,0,0,0,0,0,0,0,0,0,...,0,6,2,1,23,125,90,58,30,47
Son La,0,0,0,0,0,0,0,0,0,0,...,15,21,9,4,10,3,0,11,2,6


# General information

In [ ]:
vn_case_2m=DataFrame(vn_case_periods[vn_case_periods['Province'].isin(['Thai Binh','Quang Binh','Phu Yen','Bac Lieu','Hung Yen','Nam Dinh','Binh Thuan','Bac Giang','Son La','Hoa Binh'])])
vn_case_2m['AVG_case_per1000']=vn_case_2m['ncase_th_period']*1000/vn_case_2m['Population']
vn_case_2m

,Province,ncase_th_period,Population,area,Density(peo/sq km),AVG_case_per1000
2,Bac Giang,142,1875240,3895.89,481.338025,0.075724
4,Bac Lieu,146,918510,2667.88,344.284601,0.158953
10,Binh Thuan,2277,1246310,7942.6,156.914612,1.826993
28,Hoa Binh,4,871720,4590.3,189.904799,0.004589
29,Hung Yen,175,1284550,930.2,1380.939583,0.136234
38,Nam Dinh,30,1836270,1668.83,1100.333767,0.016337
43,Phu Yen,2562,875540,5025.96,174.203535,2.926194
44,Quang Binh,470,910660,7998.76,113.850147,0.516109
50,Son La,197,1287720,14109.83,91.264034,0.152984
52,Thai Binh,45,1875660,1584.61,1183.672954,0.023992
